In [46]:
# Import necessary libraries
import pandas as pd
import plotly.graph_objects as go
import dalmatian
from google.cloud import storage
import matplotlib.pyplot as plt
import networkx as nx
from networkx.drawing.nx_pydot import graphviz_layout

import numpy as np

import sys
import os
sys.path.append('..')  
import model_helper 
from helper import *
from plotly_helper import *

In [47]:
# Function to load data
def load_data(patient_id, wbc_file, treatment_file):
    # Load WBC and treatment data
    wbc_df = pd.read_csv(wbc_file)
    treatment_df = pd.read_csv(treatment_file, sep='\t')
    
    # Get input files from Terra through dalmatian
    workspace = 'broad-firecloud-ibmwatson/TAG_CLL_Clonal_Kinetic_UMI_PrAN'
    wm = dalmatian.WorkspaceManager(workspace)
    participants = wm.get_participants()
    
    # Load additional data for the patient
    cluster_CCF_df = pd.read_csv(participants.loc[patient_id]['cluster_ccfs'], sep='\t')
    
#     tree_df = pd.read_csv(participants.loc[patient_id]['tree_tsv'], sep='\t')
    
    tree_df = pd.read_csv(f"../Cell_Population/{patient_id}_build_tree_posteriors.tsv", sep = '\t')

    abundance_file = f"../Cell_Population/{patient_id}_cell_population_abundances.tsv"
    mcmc_file = f"../Cell_Population/{patient_id}_cell_population_mcmc_trace.tsv"  
    
    abundance_df = pd.read_csv(abundance_file, sep='\t')
    mcmc_df = pd.read_csv(mcmc_file, sep='\t')  
    
    return wbc_df, treatment_df, cluster_CCF_df, abundance_df, mcmc_df, tree_df

# Function to filter data for a specific patient
def filter_patient_data(wbc_df, patient_id):
    wbc_df_patient = wbc_df[wbc_df['Patient'] == patient_id]
    wbc_df_patient.reset_index(drop=True, inplace=True)
    times_sample = [int(i) for i in wbc_df_patient.loc[wbc_df_patient.Sample.notna(), 'Time_since_start_tx'].tolist()]
    wbc_all = [float(i) for i in wbc_df_patient['WBC'].tolist()]
    
    
    CLL_count_sample = [float(i) for i in wbc_df_patient.loc[wbc_df_patient.Sample.notna(), 'CLL count estm'].tolist()]
    all_times = [int(i) for i in wbc_df_patient['Time_since_start_tx'].to_list()]
    
    return wbc_df_patient, times_sample, CLL_count_sample, wbc_all,all_times





In [48]:
end_treatment_df = pd.read_csv("../CLL8_end_treatment.csv")

In [53]:
# Input patient information
# Input patient information
patient_id = 'CLL14-1063'
wbc_file = '../Copy of CLL14 CLL cell count 07102025 v4_Liang.csv'
treatment_file = '../GCLL_treatment.txt'
UMI_start = 1
UMI_end = 3
treatment_start = 0


In [5]:
# Manually verify the sample list order is correct

In [54]:

sample_list = ['CLL14-1063-Tumor-01-pair', 'SM-OE382', 'SM-OE383_CLL14-1063-Normal_pair', 'CLL14-1063-Tumor-02-pair'] 

In [55]:
# Output file

In [56]:
outputfile_name = "CLL14_subplot_combined_1063_wbc_updated.html"

Update patient specific information in the below cell 

In [11]:
# Get the treatment duration 

treatment_end = 365

In [12]:
treatment_end

365

In [57]:

wbc_df, treatment_df, cluster_CCF_df, abundance_df, mcmc_df, tree_df = load_data(patient_id, wbc_file,treatment_file )

# Filter data for the specific patient
wbc_df_patient, times_sample, CLL_count_sample,wbc_count, all_times = filter_patient_data(wbc_df, patient_id)

# Plot CLL count over time and save as HTML
CLL_plot_html = plot_CLL_count(patient_id, times_sample, CLL_count_sample, UMI_start, UMI_end, treatment_start, treatment_end, )
    




In [58]:
times_sample

[0, 427, 511, 518]

In [59]:
wbc_count

[55.95, 55.95, 3.97, 2.62, 4.79, 5.93, 18.55, 18.55]

In [60]:
CLL_count_sample


[55.95, 0.883317, 8.55883, nan]

In [61]:
wbc_table_html = plot_metadata_table(wbc_df_patient, patient_id )

In [62]:
tree_df

,n_iter,likelihood,edges
0,58,"[0.2, 0.25000000000000006, 0.19999999999999998...","1-2,2-3,2-4,None-1"
1,44,"[0.2, 0.25, 0.25, 0.2, 0.2, 0.25, 0.25, 0.2, 0...","1-2,1-3,1-4,None-1"
2,39,"[0.25000000000000006, 0.3333333333333333, 0.25...","1-2,2-3,3-4,None-1"
3,39,"[0.19999999999999998, 0.25, 0.25, 0.2500000000...","1-2,1-4,2-3,None-1"
4,39,"[0.3333333333333333, 0.25, 0.25, 0.25, 0.25000...","1-2,1-3,3-4,None-1"
5,30,"[0.2, 0.25, 0.25, 0.24999999999999994, 0.25, 0...","1-2,1-3,2-4,None-1"


In [63]:
 plot_tree_html = plot_tree_plotly(tree_df, 4)

In [64]:
 plot_ccf_html = plot_ccf(cluster_CCF_df, times_sample, treatment_df,)

In [65]:
branch_annotations = {
    1: "MAP2K2",
    3:"ASXL2"
    
}

In [94]:
ccf_tree_html = plot_ccf_tree_combined(
    tree_df=tree_df,
    tree_selected=4,
    ccf_df=cluster_CCF_df,
    times_sample=times_sample,
    treatment_df=treatment_df,
    branch_annotations=branch_annotations

)

In [67]:
cluster_list, cluster_abundance = model_helper.get_abundance(abundance_df, mcmc_df, sample_list)
subclone_sample, log_subclone_sample = model_helper.calc_subclone(CLL_count_sample, cluster_abundance, cluster_list)
all_abundance = model_helper.get_all_abundance(cluster_list, mcmc_df, sample_list, times_sample, )

In [68]:
#  Add noise to all abundance

import random 
random.seed(42)
def add_uniform_noise_and_normalize(lst, low=0, high=0.01):
    
    # Add uniform noise
    noisy_lst = [x + random.uniform(low, high) for x in lst]

    
    # Normalize the list
    total_sum = sum(noisy_lst)
    
    normalized_lst = [x / total_sum for x in noisy_lst]

    return normalized_lst


subclone_cluster_iter = {}
# Iterate through each cluster
for cluster, iterations in all_abundance.items():
   
    # Iterate through each iteration in the cluster
    for iteration, value in iterations.items():
        # If the iteration is not in the new dict, add it
        if iteration not in subclone_cluster_iter:
            subclone_cluster_iter[iteration] = {}
        # Add the cluster and its value to this iteration
        subclone_cluster_iter[iteration][cluster] = value

        
        
noise_added_iter = {}

for iteration, abundances in subclone_cluster_iter.items():
    
    transposed_data = {i: list(values) for i, values in enumerate(zip(*abundances.values()), start=1)}
    
    noise_added_iter[iteration] = {}
    for time_point, abundance in transposed_data.items():
        
#         print(time_point, abundance)
        noise_abundance = add_uniform_noise_and_normalize(abundance)
    
#         print(noise_abundance)

        noise_added_iter[iteration][time_point] = add_uniform_noise_and_normalize(abundance)

    
noise_added_all_abundance = {}
for iteration, abundances in noise_added_iter.items():
    original_format = {i + 1: list(values) for i, values in enumerate(zip(*abundances.values()))}
    noise_added_all_abundance[iteration] = original_format
    
    
noise_added_original_all_abundances = {}

for iteration, abundances in noise_added_all_abundance.items():
    
    for cluster, abundance_value in abundances.items():
        if cluster not in noise_added_original_all_abundances:
            noise_added_original_all_abundances[cluster] = {}
        noise_added_original_all_abundances[cluster][iteration] = abundance_value
            

In [69]:
subclone_sample_mcmc_with_uniform_noise, log_subclone_sample_mcmc_with_uniform_noise = model_helper.calc_subclone(CLL_count_sample, noise_added_original_all_abundances, cluster_list, input_type = "mcmc")

In [70]:
times_aft_tx = [x for x in all_times if x > treatment_end]
times_aft_tx.insert(0, treatment_end)

# Set the extrapolate time after treatment
extrapolate_start_idx = 1

In [71]:
times_sample

[0, 427, 511, 518]

In [72]:
times_sample[extrapolate_start_idx:]

[427, 511, 518]

In [73]:
# Generate Plotly plot
subclone_plot_html = plot_subclones(cluster_list, times_sample, CLL_count_sample, log_subclone_sample, extrapolate_start_idx, times_aft_tx, treatment_df, treatment_end)


In [74]:
linear_model_mcmc_html = plot_linear_model_mcmc(cluster_list,times_sample,  CLL_count_sample, log_subclone_sample_mcmc_with_uniform_noise,extrapolate_start_idx,times_aft_tx,treatment_df, treatment_end)  

In [75]:
# Convert the column "CLL count estm" to float 
wbc_df_patient['CLL count estm'] = wbc_df_patient['CLL count estm'].astype(float) 
times_sliced_aft = [int(i) for i in wbc_df_patient[wbc_df_patient['CLL count estm'] > 0]['Time_since_start_tx'].values  if int(i) > 0]

/var/folders/x6/7zg8fddx4dlgx2lv4sv34_g40000gp/T/ipykernel_24074/981427378.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [76]:
times_aft_tx

[365, 427, 511, 518]

In [77]:
# Needs to manually check times_sliced_aft

In [78]:
times_sliced_aft

[168, 252, 308, 427, 511]

In [79]:
len(times_sliced_aft)

5

In [80]:
log_subclone_sample_mcmc_with_uniform_noise

{1: {0: [2.0164465817132076, -3.6227032969712893, -2.6813236319058733, nan],
  1: [2.1631069555416844, -3.9790686338444536, -3.755999580018857, nan],
  2: [2.1414642696329804, -3.6747107063015445, -3.34270103236842, nan],
  3: [1.7245099352269184, -5.512723597914225, -2.206086869409112, nan],
  4: [1.9519208830432742, -3.4823120294333556, -4.033716121759857, nan],
  5: [2.1102752439268433, -3.8224447618362367, -1.163848236427278, nan],
  6: [1.9454820331911789, -3.4892153531722334, -3.2024419932132897, nan],
  7: [1.9832987618816367, -3.6679173463058894, -4.004050397314618, nan],
  8: [1.8043452152488633, -3.206961385287192, -4.599239531379943, nan],
  9: [1.138604545841754, -4.807969722345303, -2.700666129140304, nan],
  10: [1.0354163348827563, -4.971599696907713, -2.647042631644018, nan],
  11: [2.038839810596089, -3.2579186426526228, -2.721810960888785, nan],
  12: [1.3272490128644594, -3.5050018800839777, -2.679420971864305, nan],
  13: [1.9972145354238429, -5.0029302987838244, -1

In [85]:
index_samples_model = slice(1,4)

# index_samples_model is the index of the samples after treatment. For instance, slice(1,7) means sample 2, 3, 4, 5,6, 7

X, y = create_inputs(times_sliced_aft, log_subclone_sample_mcmc_with_uniform_noise,2, index_samples_model, times_sample)



In [86]:
y

[None,
 None,
 None,
 array([-3.67471071, -0.88783724, -1.27361989, -1.79004988]),
 array([-3.34270103,  1.5620663 ,  0.49540492,  0.75004012])]

In [87]:
wbc_aft_tx = wbc_df_patient[wbc_df_patient.Time_since_start_tx > treatment_end]['CLL count estm'].to_list()
wbc_model = [i for i in wbc_aft_tx if i > 0]

In [88]:
wbc_model

[0.883317, 8.55883]

In [89]:
len(wbc_model)

2

In [90]:
n_clusters = len(cluster_list)
logsumexp_points = np.log(wbc_model)
model = MultiClusterLinearRegression(n_clusters, X, y)
model.fit(logsumexp_points)

In [91]:
# Generate Plotly plot
subclone_plot_new_model_html = plot_subclones_new_model(cluster_list, times_sample, wbc_model, log_subclone_sample, extrapolate_start_idx, times_aft_tx, times_sliced_aft, treatment_df,treatment_end, model )


In [92]:
plot_mcmc_new_model_html = plot_mcmc_model(cluster_list, index_samples_model, times_aft_tx, times_sample, times_sliced_aft, sample_list, wbc_model, log_subclone_sample_mcmc_with_uniform_noise, treatment_df, treatment_end)


In [95]:
create_html_file([wbc_table_html, plot_ccf_html, ccf_tree_html, CLL_plot_html, subclone_plot_html,linear_model_mcmc_html,subclone_plot_new_model_html, plot_mcmc_new_model_html], output_file=outputfile_name)

HTML file saved as CLL14_subplot_combined_1063_wbc_updated.html
